(notebook réalisé sur colab lié à un google drive) 

Les fichiers prêts (output de ce notebook) à disposition :

Lien de téléchargement du fichier full_xenocanto.csv : 
https://drive.google.com/file/d/1IqIknH0h5GigT3TWyE8SuFCUk6q-wbLW/view?usp=sharing

Lien de téléchargement de rand_id.csv : 
https://drive.google.com/file/d/1-1CcgTUS1jp8qmVW2yLVELM9GF54jKtq/view?usp=sharing

Lien de téléchargement des fichiers mp3 depuis google_drive : 
https://drive.google.com/drive/folders/1VMF_Y-ABxHfnBqIDR5rc5TDpgIFxQDJ_?usp=sharing

In [1]:
import os
import pandas as pd
import json
import urllib
import numpy as np

# Creation de full_xenocanto.csv (tous les enregistrements, 23953)

In [3]:
# loads the list of xenocanto.com records to full_xeno.csv

birds_path = 'https://raw.githubusercontent.com/microfaune/microfaune/master/oiseaux.csv'
birds = pd.read_csv(birds_path, delimiter = ';')

full_xeno = pd.DataFrame()

for i in range(birds.shape[0]):
    bird_query = birds.iloc[i]["Nom latin"].replace(" ", "+").lower()
    bird_name = birds.iloc[i]["Nom latin"].replace(" ", "_").lower()
    
    url = f'https://www.xeno-canto.org/api/2/recordings?query={bird_query}'
    r = urllib.request.urlopen(url)
    
    bird_data = json.loads(r.read().decode(r.info().get_param('charset') or 'utf-8'))
    
    full_xeno = full_xeno.append(pd.json_normalize(bird_data['recordings']))

full_xeno.insert(loc = 1, column = 'species', value = full_xeno['gen'] + '_' + full_xeno['sp'])

full_xeno.to_csv(r'/content/drive/My Drive/full_xenocanto.csv', index = False)

In [130]:
full_xeno.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23953 entries, 0 to 499
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             23953 non-null  object
 1   species        23953 non-null  object
 2   gen            23953 non-null  object
 3   sp             23953 non-null  object
 4   ssp            23953 non-null  object
 5   en             23953 non-null  object
 6   rec            23953 non-null  object
 7   cnt            23953 non-null  object
 8   loc            23953 non-null  object
 9   lat            23766 non-null  object
 10  lng            23765 non-null  object
 11  alt            23953 non-null  object
 12  type           23953 non-null  object
 13  url            23953 non-null  object
 14  file           23953 non-null  object
 15  file-name      23953 non-null  object
 16  lic            23953 non-null  object
 17  q              23953 non-null  object
 18  length         23953 non-nul

In [38]:
full_xeno['id'].unique().size

23953

# Chargement d'un full_xenocanto.csv déjà existant

In [157]:
# loading full_xeno from the already existing full_xeno.csv
full_xeno = pd.read_csv('/content/drive/My Drive/full_xenocanto.csv') # le 26/10/2020 23948 enregistrements

In [118]:
full_xeno["species"].unique().size

52

# Création dataframe avec la liste des espèces (52)

In [69]:
# getting species list 
species_list = full_xeno.groupby('species', as_index=False).count().sort_values(by  = 'id', ascending = True, ignore_index = True)

In [158]:
species_list["species"].size

52

# Création de rand_id.csv : sampling de 138 enregistrements par espèce
(7176 enregistrements au total)

In [147]:
# sample examples from full_xeno
rand_id = pd.DataFrame()

for species in species_list['species'] : 
  rand_id = rand_id.append(full_xeno[full_xeno["species"]  == species].sample(n = 138)) # 138 examples (l'espece avec le moins d'examples)

In [154]:
rand_id.shape

(7176, 30)

In [149]:
rand_id["species"].unique().size

52

In [152]:
rand_id.head()

,id,species,gen,sp,ssp,en,rec,cnt,loc,lat,lng,alt,type,url,file,file-name,lic,q,length,time,date,uploaded,also,rmk,bird-seen,playback-used,sono.small,sono.med,sono.large,sono.full
0,561492,Falco_subbuteo,Falco,subbuteo,,Eurasian Hobby,Albert Lastukhin,Russian Federation,"Krasnoarmeysky District, Chuvashia",55.8196,46.9725,160,"adult, alarm call, female, flight call, male",//www.xeno-canto.org/561492,//www.xeno-canto.org/561492/download,"XC561492-Falco subbuteo 0506_090515, B1.MP3",//creativecommons.org/licenses/by-nc-sa/4.0/,B,0:49,09:00,2020-05-06,2020-05-24,[Corvus corax],,yes,no,//www.xeno-canto.org/sounds/uploaded/LELYWQKUZ...,//www.xeno-canto.org/sounds/uploaded/LELYWQKUZ...,//www.xeno-canto.org/sounds/uploaded/LELYWQKUZ...,//www.xeno-canto.org/sounds/uploaded/LELYWQKUZ...
1,495035,Falco_subbuteo,Falco,subbuteo,,Eurasian Hobby,Albert Lastukhin,Russian Federation,"Gornomariyskiy Rayon, Mari El Republic",56.1832,46.1057,60,call,//www.xeno-canto.org/495035,//www.xeno-canto.org/495035/download,"XC495035-Falco subbuteo 11-12.05.2018, Хмелёвк...",//creativecommons.org/licenses/by-nc-sa/4.0/,B,0:21,05:30,2018-05-12,2019-08-31,[],,yes,no,//www.xeno-canto.org/sounds/uploaded/LELYWQKUZ...,//www.xeno-canto.org/sounds/uploaded/LELYWQKUZ...,//www.xeno-canto.org/sounds/uploaded/LELYWQKUZ...,//www.xeno-canto.org/sounds/uploaded/LELYWQKUZ...
2,589140,Falco_subbuteo,Falco,subbuteo,,Eurasian Hobby,Byoungsoon Jang,South Korea,"Gunsan, Jeollabuk-do",35.9259,126.7455,20,call,//www.xeno-canto.org/589140,//www.xeno-canto.org/589140/download,XC589140-200917_0426.15.33(eurasian hobby).mp3,//creativecommons.org/licenses/by-nc-sa/4.0/,C,0:02,16:30,2020-09-17,2020-09-17,[],,yes,no,//www.xeno-canto.org/sounds/uploaded/RVLNPQSYV...,//www.xeno-canto.org/sounds/uploaded/RVLNPQSYV...,//www.xeno-canto.org/sounds/uploaded/RVLNPQSYV...,//www.xeno-canto.org/sounds/uploaded/RVLNPQSYV...
3,262833,Falco_subbuteo,Falco,subbuteo,,Eurasian Hobby,Albert Lastukhin,Russian Federation,"Kozlovskiy rayon, Chuvashia",55.8697,48.0544,50,flight call,//www.xeno-canto.org/262833,//www.xeno-canto.org/262833/download,"XC262833-Falco 150719_001,AA.MP3",//creativecommons.org/licenses/by-nc-sa/4.0/,B,0:09,05:00,2015-07-19,2015-07-20,[],,unknown,no,//www.xeno-canto.org/sounds/uploaded/LELYWQKUZ...,//www.xeno-canto.org/sounds/uploaded/LELYWQKUZ...,//www.xeno-canto.org/sounds/uploaded/LELYWQKUZ...,//www.xeno-canto.org/sounds/uploaded/LELYWQKUZ...
4,485666,Falco_subbuteo,Falco,subbuteo,,Eurasian Hobby,Stanislas Wroza,Russian Federation,"Kabansky District, Buryatia",52.0916,106.9924,480,"flight call, interaction calls",//www.xeno-canto.org/485666,//www.xeno-canto.org/485666/download,XC485666-2019-06-23 Selenga to Ulan Ude Faucon...,//creativecommons.org/licenses/by-nc-sa/4.0/,D,0:05,13:00,2019-06-23,2019-07-09,[],,yes,no,//www.xeno-canto.org/sounds/uploaded/SDPCHKOHR...,//www.xeno-canto.org/sounds/uploaded/SDPCHKOHR...,//www.xeno-canto.org/sounds/uploaded/SDPCHKOHR...,//www.xeno-canto.org/sounds/uploaded/SDPCHKOHR...


In [151]:
rand_id.reset_index(drop = True, inplace = True)

In [160]:
rand_id.to_csv(r'/content/drive/My Drive/rand_id.csv')

# Chargement d'un rand_id.csv pré-existant

In [ ]:
rand_id = pd.read_csv('/content/drive/My Drive/rand_id.csv')

# Download des enregistrements de rand_id.csv

In [155]:
# Downloading examples
for i in range (rand_id.shape[0]) :
  bird = rand_id.iloc[i]["species"]
  filepath = rand_id.iloc[i]["file"]
  filename = rand_id.iloc[i]["file-name"]

 # create folder if it doesn't exist
  dirPath = f'/content/drive/My Drive/xenocanto_data/{bird}'
    
  if not os.path.exists(dirPath):
    os.makedirs(dirPath)
    
    # retrieve recordings
  rec_path = f"https:{filepath}"
       
  urllib.request.urlretrieve(rec_path, f"/content/drive/My Drive/xenocanto_data/{bird}/{filename}")